In [53]:
import selenium
import time #time.sleep에 필요
import pymysql

#db = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='1234', db='ohmy', charset='utf8')
#cursor = db.cursor()

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from datetime import datetime # 날짜 관련 라이브러리
from datetime import datetime
from datetime import timedelta
from dateutil import relativedelta

# 아래부터는 뤱페이지 로딩 대기를 위해 필요한 라이브러리
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = webdriver.ChromeOptions()
options.add_argument("user_agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.47")
driver = webdriver.Chrome(options=options)
driver.get('https://www.kopis.or.kr/por/stats/perfo/perfoStatsTotal.do?menuId=MNU_000135&searchWord=&searchType=total')

# getMonthRange 함수 개요:
# year와 month를 입력받는다 (2019, 01)
# 해당 month의 시작일자와 마지막일자를 구한다. (1월31일, 2월28일....11월30일, 12월31일)
# yyyy.mm.dd 형식으로 리턴 (2019.01.01, 2019.01.31)
def getMonthRange(year, month): 
    year = int(year)
    month = int(month)
    this_month = datetime(year=year, month=month, day=1).date()
    next_month = this_month + relativedelta.relativedelta(months=1)
    first_day = this_month
    last_day = next_month - timedelta(days=1)
    month__=format(first_day.month,'02')
    month__day=month__+format(first_day.day,'02')
    day_left= '{}{}'.format(year,month__day)
    month__ = format(first_day.month, '02')
    month__day = month__ + format(last_day.day, '02')
    day_right = '{}{}'.format(year, month__day)
    return day_left, day_right

# make_year_month 함수 개요:
# 시작연도 입력시 현재 연+월 에서 1달을 뺀 전체 데이터를 리턴
# ex) 2019 입력시 2019년01월부터 현재날짜인 2022년11월 바로 전달까지 리턴 (2019.01~~~~2022.10)
def make_year_month(inyear) :
    nowyear = datetime.now().year
    nowmonth = datetime.now().month
    year_month=[]
#     for year in range(startyear,nowyear) :
#         for month in range (1,13) :
#             year_month.append((year,month))
#             print(year_month)
    year = inyear
    if int(year) == int(nowyear):
        for month in range(1, nowmonth) :
            year_month.append((nowyear, month))
    else:
        for month in range (1,13) :
            year_month.append((year, month))

    return year_month


# searchData 함수 개요(사이트 에서 데이터를 스크래핑 해서 가져옴) : 
# 검색 범위를 입력받는다 (dayleft, dayright) ex) 2019.01.01, 2019.01.31
# selenium 으로 웹페이지를 컨트롤해서 조회 버튼 클릭
# 결과로 나온 데이터 테이블을 crwaling 해서 result 변수에 담는다.
# result 변수를 리턴해준다.
def searchData(dayleft, dayright) : 
    elem = driver.find_element_by_id('startDate') #검색 시작 날짜
    elem.send_keys(Keys.CONTROL + 'a') # ctrl + a : 커서가 올라간 범위내 모든 텍스트 선택
    elem.send_keys(Keys.BACKSPACE) # backspace 키 입력으로 선택된 부분 clear
    elem.send_keys(dayleft) # 검색시작일자 입력 (2019.01.01)
    elem.send_keys(Keys.ENTER)
    time.sleep(1) # 엔터키 누르고 바로 넘어가면 다음 입력이 안되는 경우가 있어서 1초간 대기

    elem = driver.find_element_by_id('endDate') #검색 마지막 날짜
    elem.send_keys(Keys.CONTROL + 'a')
    elem.send_keys(Keys.BACKSPACE)
    elem.send_keys(dayright)
    elem.send_keys(Keys.ENTER)
    time.sleep(1)

    elem = driver.find_element_by_xpath('//*[@id="search_list"]/div/a') # 클릭버튼 xpath
    elem.click()
    
    # 페이지 로딩을 기다리는 내장함수, 로딩 아이콘이 사라질때 까지 대기
    wait = WebDriverWait(driver, 60).until(
        EC.invisibility_of_element_located((By.XPATH, '//*[@id="progress"]'))
    )

    table = driver.find_element_by_xpath('//*[@id="su_con"]/div[5]/table') # 결과 테이블 xpath
    tbody = table.find_element_by_tag_name('tbody') # 테이블구조 : table > thead, tbody, tfoot> tr > td
    result = ''                                     # 우리가 필요한 데이터는 tbody 에 존재하므로 tbody 선택
    
    for tr in tbody.find_elements_by_class_name("trTotal"): # 테이블 tbody 에 있는 trTotal 행 선택
        txt = tr.find_element_by_tag_name("th").get_attribute("innerText") # th는 가장 좌측에 있는 권역이름(서울, 경기도....)
        txt += ' ' + dayleft + ' ' + dayright
        for td in tr.find_elements_by_tag_name("td")[1:]: # td[0] = 합계, td[1] = 공연건수(건), td[2] = 공연건수(%), td[3] = 개막편수
            txt += ' ' + td.get_attribute("innerText").replace(',', '') # innertext는 괄호와 괄호 사이에 있는 텍스트를 읽는다 <td> innerText </td> 
        result += txt + '\n' # th + td[1], td[2].... td[마지막] 까지 문자열 데이터를 붙여서 결과 함수에 담는다. '\n'은 줄바꿈
    
    return result

# make_dict_send_query 함수 개요 (가져온 데이터를 가공하는 함수):
# searchData 함수에서 받은 result 배열을 dictionary 형태로 가공
# dictionary 형태로 가공한 data를 createInsertQuery 을 통해 INSERT 구문을 생성해서
# db에 insert
def make_dict_send_query(result, tablename) :
    for i in range(0, len(result)):
        td = result[i]
        td = td.split()
        #print(td)

        location = td[0]
        start_date = td[1]
        end_date = td[2]
        perf_count = td[3]
        perf_per = td[4]
        opening_count = td[5]
        opening_per = td[6]
        play_count = td[7]
        play_per = td[8]
        ticket_sale_count = td[9]
        ticket_sale_per = td[10]
        ticket_sale_amt = td[11]
        ticket_sale_amt_per = td[12] 
        data = {
            'location' : location
            , 'start_date' : start_date
            , 'end_date' : end_date
            , 'perf_count' : perf_count
            , 'perf_per' : perf_per
            , 'opening_count' : opening_count
            , 'opening_per' : opening_per
            , 'play_count' : play_count
            , 'play_per' : play_per
            , 'ticket_sale_count' : ticket_sale_count
            , 'ticket_sale_per' : ticket_sale_per
            , 'ticket_sale_amt' : ticket_sale_amt
            , 'ticket_sale_amt_per' : ticket_sale_amt_per
        }
        query = createInsertQuery(tablename, data)
        cursor.execute(query)
        
# createInsertQuery 함수 개요:
# db table 이름과 dictionary {컬럼명 : 밸류} 형태의 데이터를 입력받아 insert 쿼리문 반환
# dictionary 데이터 key값은 db컬럼명, value값은 row데이터
# 조인문으로 각 값들 사이에 ,(쉼표) 추가
# value 값이 숫자인지 str인지 체크
# INSERT INTO 테이블이름 ('컬렴명1', '컬럼명2'...) VALUES {value1, value2.....} 형태로 반환
def createInsertQuery(tablename : str, data : dict):
    sql= 'insert into ' + tablename + ' ({}) values ({})'
    key = ', '.join(list(data.keys()))
    value = ', '.join(str(e) if isNumber(str(e)) else '"'+str(e)+'"' for e in data.values())
    return(sql.format(key, value))

# INSERT INTO (test) + `location`, 'start_date`, .....' VALUES value1, value2...  ''
#  data = {
#             'location' : location
#             , 'start_date' : start_date
#             , 'end_date' : end_date
#             , 'perf_count' : perf_count
#             , 'perf_per' : perf_per
#             , 'opening_count' : opening_count
#             , 'opening_per' : opening_per
#             , 'play_count' : play_count
#             , 'play_per' : play_per
#             , 'ticket_sale_count' : ticket_sale_count
#             , 'ticket_sale_per' : ticket_sale_per
#             , 'ticket_sale_amt' : ticket_sale_amt
#             , 'ticket_sale_amt_per' : ticket_sale_amt_per
#         }

# isNumber 함수 개요:
# 값을 입력받아서 숫자형(float, int)을 검사
# float 또는 int형일 경우 true / 아닐경우 false 리턴
def isNumber(n):
    if n != 'NULL':
        try:
            float(n) or int(n)
        except ValueError:
            return False

    return True

# 테이블 생성하는 함수
# 테이블명이 존재하면 생성 안함
# 이 때 발생하는 exception 오류로 인해 스크립트가 멈추는걸 방지하기 위해 try문 사용
def create_table(tablename):
    sql='''
    create table if not exists {}(
     seq          	 		int NOT NULL AUTO_INCREMENT PRIMARY KEY,
     location        		varchar(10),
     start_date				varchar(10),
     end_date				varchar(10),
     perf_count      		int,
     perf_per    			float,
     opening_count    	 	int,
     opening_per        	float,
     play_count      		int,
     play_per    			float,
     ticket_sale_count      INT,
     ticket_sale_per   		float,
     ticket_sale_amt     	BIGINT,
     ticket_sale_amt_per    float
); 
    '''.format(tablename)
    try:
        cursor.execute(sql)
        db.commit()
    except:
        return False
   
    return True

def is_valid_datetime(inyear) :
    regex = datetime.strptime
    try :
        regex(inyear, '%Y')
        # regex(_date, '%Y-%m-%d %H:%M:%S')

    except ValueError:
        return False

    return True

In [61]:
# 프로젝트 시연 부분

db = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')

# 2019
year = input()

if int(year) > datetime.now().year:
    print('err')
else:
    tablename = 'kopis_data'
    cursor = db.cursor()
    create_table(tablename)
    year_month_list = make_year_month(year)

    for ym in year_month_list :
        dayleft, dayright = getMonthRange(ym[0], ym[1])
        result = searchData(dayleft, dayright)
        result = result.strip().split('\n')
        make_dict_send_query(result, tablename)
        
    db.commit()    
    db.close()
    print('search done')
    driver.close()
    


2033
err


In [62]:
# 2019년07월 ~ 2019년 12월 월별집계 (2019 하반기)  
db = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='1234', db='mywork', charset='utf8')
sql = '''
select mid(start_date, 1, 6) as period, sum(ticket_sale_count) count, sum(ticket_sale_amt) sale_amt
	from kopis_data 
	where mid(start_date, 1, 6) like '2019%'
		and quarter(start_date) in(3, 4)
	group by period with rollup;
'''
cursor = db.cursor()
cursor.execute(sql)
result = cursor.fetchall()
db.close()
for data in result:
    print(data)


('201907', Decimal('737597'), Decimal('17562173'))
('201908', Decimal('1143359'), Decimal('28169107'))
('201909', Decimal('866918'), Decimal('23995988'))
('201910', Decimal('1112859'), Decimal('31113594'))
('201911', Decimal('1140366'), Decimal('35436416'))
('201912', Decimal('1633033'), Decimal('56484685'))
(None, Decimal('6634132'), Decimal('192761963'))


In [49]:
# 2020년 01월 ~ 2020년 06월 집계 (2020 상반기)
sql = '''
select mid(start_date, 1, 6) as period, sum(ticket_sale_count) count, sum(ticket_sale_amt) sale_amt
	from kopis_data 
	where mid(start_date, 1, 6) like '2020%'
		and quarter(start_date) in(1, 2)
	group by period with rollup;
'''
db = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='asdf12', db='mywork', charset='utf8')
cursor = db.cursor()
cursor.execute(sql)
result = cursor.fetchall()
db.close()
for data in result:
    print(data)

('202001', Decimal('1007409'), Decimal('40502460'))
('202002', Decimal('542193'), Decimal('21985485'))
('202003', Decimal('162233'), Decimal('9188961'))
('202004', Decimal('115838'), Decimal('4693198'))
('202005', Decimal('250702'), Decimal('11513004'))
('202006', Decimal('234836'), Decimal('10411442'))
(None, Decimal('2313211'), Decimal('98294550'))


In [52]:
# 2019년 01월~ 2022년 12월 분기별 합계
sql = '''
select year(start_date) years, quarter(start_date) quarters, sum(ticket_sale_amt)*1000 sale_amt,
sum(ticket_sale_count) sum_count_amt
# round(sum(ticket_sale_amt) /sum(ticket_sale_count)*1000,0) perticket
from kopis_data
where  year(start_date) >= 2019 and mid(start_date, 1, 6) <= 202209
group by 1,2 with rollup;
'''
db = pymysql.connect(host='127.0.0.1', port=3306, user='root', passwd='asdf12', db='mywork', charset='utf8')
cursor = db.cursor()
cursor.execute(sql)
result = cursor.fetchall()
db.close()
for data in result:
    print(data)

(2019, 1, Decimal('20737887000'), Decimal('830623'))
(2019, 2, Decimal('26198114000'), Decimal('1145827'))
(2019, 3, Decimal('69727268000'), Decimal('2747874'))
(2019, 4, Decimal('123034695000'), Decimal('3886258'))
(2019, None, Decimal('239697964000'), Decimal('8610582'))
(2020, 1, Decimal('71676906000'), Decimal('1711835'))
(2020, 2, Decimal('26617644000'), Decimal('601376'))
(2020, 3, Decimal('40819162000'), Decimal('917504'))
(2020, 4, Decimal('32952779000'), Decimal('936090'))
(2020, None, Decimal('172066491000'), Decimal('4166805'))
(2021, 1, Decimal('41764570000'), Decimal('848333'))
(2021, 2, Decimal('75498399000'), Decimal('1715402'))
(2021, 3, Decimal('72401569000'), Decimal('1761862'))
(2021, 4, Decimal('117308611000'), Decimal('2779950'))
(2021, None, Decimal('306973149000'), Decimal('7105547'))
(2022, 1, Decimal('214253504000'), Decimal('4363932'))
(2022, 2, Decimal('248618764000'), Decimal('6366596'))
(2022, 3, Decimal('282133940000'), Decimal('7508928'))
(2022, None, Dec

코로나 타격 공연계 온라인예매 매출 감소 관람객 감소

2019년 하반기 꾸준한 온라인 예매금액 상향
2020년 상반기 코로나 유행 및 2020년 3월 사회적 거리두기 시행
    사회적 거리두기는 위험단계를 1단계부터 3단계까지 구분하여 당기의
    상황을 고려하여 시행되었으나 위험단계와 무관하게 사회적 거리두기의 여파로 인해 대부분의
    공연이 취소되거나 무기한 연기되어 큰 타격
    
2020년 사회적 거리두기 초장기(가장 강화되어있을때) 약간 감소
but 비대면활성화로 인한 온라인 집계의 양이 증가
온라인 공연의 활성화
백신보급 및 거리두기 완화 이후 지속적인 관람객 증가

2019년 12월 코로나19 유행
2020년 3월 사회적거리두기 시작
2021년 9월 전국민 백신 1차 80%
2021년 12월 전국민 백신 2차 80%

감염 확산세가 급진적으로 증가함에 따라 방역 당국은 지역사회 감염 차단을 위해 사회적
거리두기라는 권고수칙을 시행하게 되었는데 이는 행사와 모임, 외출을 자제하고 재택근무의
확대 등의 행동강령으로 사람들의 집단 감염 가능성을 최소화해 지역사회의 감염을 예방하는
것에 목적을 두고 있다. 사회적 거리두기는 위험단계를 1단계부터 3단계까지 구분하여 당기의
상황을 고려하여 시행되었으나 위험단계와 무관하게 사회적 거리두기의 여파로 인해 대부분의
상권이 마비되어 존폐의 기로에 서게 되는 등의 부작용들이 발생하기 시작했다. 방